# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 22 2023 12:55PM,257964,10,Yusen-9192319,Yusen – 3D Matrix,Released
1,Feb 22 2023 12:55PM,257964,10,Yusen-9192321,Yusen – 3D Matrix,Released
2,Feb 22 2023 12:54PM,257963,10,PRF-51043-BO,Bio-PRF,Released
3,Feb 22 2023 12:53PM,257962,10,Enova-11396,Emerginnova,Released
4,Feb 22 2023 12:53PM,257962,10,Enova-11397,Emerginnova,Released
5,Feb 22 2023 12:53PM,257962,10,Enova-11398,Emerginnova,Released
6,Feb 22 2023 12:52PM,257961,10,MSP218479,"Methapharm, Inc.",Released
7,Feb 22 2023 12:52PM,257961,10,MSP218610,"Methapharm, Inc.",Released
8,Feb 22 2023 12:52PM,257961,10,MSP218804,"Methapharm, Inc.",Released
9,Feb 22 2023 12:52PM,257961,10,MSP218795,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,257960,Released,7
36,257961,Released,9
37,257962,Released,3
38,257963,Released,1
39,257964,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257960,NaN,7.0
257961,NaN,9.0
257962,NaN,3.0
257963,NaN,1.0
257964,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257858,1.0,0.0
257885,0.0,14.0
257886,0.0,1.0
257889,0.0,2.0
257890,57.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257858,1,0
257885,0,14
257886,0,1
257889,0,2
257890,57,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257858,1,0
1,257885,0,14
2,257886,0,1
3,257889,0,2
4,257890,57,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257858,1,
1,257885,,14
2,257886,,1
3,257889,,2
4,257890,57,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 22 2023 12:55PM,257964,10,Yusen – 3D Matrix
2,Feb 22 2023 12:54PM,257963,10,Bio-PRF
3,Feb 22 2023 12:53PM,257962,10,Emerginnova
6,Feb 22 2023 12:52PM,257961,10,"Methapharm, Inc."
15,Feb 22 2023 12:44PM,257960,10,"Methapharm, Inc."
19,Feb 22 2023 12:44PM,257960,10,Methapharm-G
22,Feb 22 2023 12:38PM,257959,10,ISDIN Corporation
30,Feb 22 2023 12:11PM,257958,16,Sartorius Bioprocess Solutions
31,Feb 22 2023 12:11PM,257957,10,"Digital Brands, LLC"
32,Feb 22 2023 12:09PM,257956,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 22 2023 12:55PM,257964,10,Yusen – 3D Matrix,,2
1,Feb 22 2023 12:54PM,257963,10,Bio-PRF,,1
2,Feb 22 2023 12:53PM,257962,10,Emerginnova,,3
3,Feb 22 2023 12:52PM,257961,10,"Methapharm, Inc.",,9
4,Feb 22 2023 12:44PM,257960,10,"Methapharm, Inc.",,7
5,Feb 22 2023 12:44PM,257960,10,Methapharm-G,,7
6,Feb 22 2023 12:38PM,257959,10,ISDIN Corporation,,8
7,Feb 22 2023 12:11PM,257958,16,Sartorius Bioprocess Solutions,,1
8,Feb 22 2023 12:11PM,257957,10,"Digital Brands, LLC",,1
9,Feb 22 2023 12:09PM,257956,19,"GCH Granules USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 22 2023 12:55PM,257964,10,Yusen – 3D Matrix,2,
1,Feb 22 2023 12:54PM,257963,10,Bio-PRF,1,
2,Feb 22 2023 12:53PM,257962,10,Emerginnova,3,
3,Feb 22 2023 12:52PM,257961,10,"Methapharm, Inc.",9,
4,Feb 22 2023 12:44PM,257960,10,"Methapharm, Inc.",7,
5,Feb 22 2023 12:44PM,257960,10,Methapharm-G,7,
6,Feb 22 2023 12:38PM,257959,10,ISDIN Corporation,8,
7,Feb 22 2023 12:11PM,257958,16,Sartorius Bioprocess Solutions,1,
8,Feb 22 2023 12:11PM,257957,10,"Digital Brands, LLC",1,
9,Feb 22 2023 12:09PM,257956,19,"GCH Granules USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 22 2023 12:55PM,257964,10,Yusen – 3D Matrix,2,
1,Feb 22 2023 12:54PM,257963,10,Bio-PRF,1,
2,Feb 22 2023 12:53PM,257962,10,Emerginnova,3,
3,Feb 22 2023 12:52PM,257961,10,"Methapharm, Inc.",9,
4,Feb 22 2023 12:44PM,257960,10,"Methapharm, Inc.",7,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 22 2023 12:55PM,257964,10,Yusen – 3D Matrix,2.0,NaN
1,Feb 22 2023 12:54PM,257963,10,Bio-PRF,1.0,NaN
2,Feb 22 2023 12:53PM,257962,10,Emerginnova,3.0,NaN
3,Feb 22 2023 12:52PM,257961,10,"Methapharm, Inc.",9.0,NaN
4,Feb 22 2023 12:44PM,257960,10,"Methapharm, Inc.",7.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 22 2023 12:55PM,257964,10,Yusen – 3D Matrix,2.0,0.0
1,Feb 22 2023 12:54PM,257963,10,Bio-PRF,1.0,0.0
2,Feb 22 2023 12:53PM,257962,10,Emerginnova,3.0,0.0
3,Feb 22 2023 12:52PM,257961,10,"Methapharm, Inc.",9.0,0.0
4,Feb 22 2023 12:44PM,257960,10,"Methapharm, Inc.",7.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5932432,107.0,1.0
12,257954,0.0,1.0
15,257890,12.0,57.0
16,515913,2.0,0.0
18,257920,0.0,1.0
19,1547526,6.0,3.0
21,515891,2.0,0.0
22,515811,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5932432,107.0,1.0
1,12,257954,0.0,1.0
2,15,257890,12.0,57.0
3,16,515913,2.0,0.0
4,18,257920,0.0,1.0
5,19,1547526,6.0,3.0
6,21,515891,2.0,0.0
7,22,515811,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,107.0,1.0
1,12,0.0,1.0
2,15,12.0,57.0
3,16,2.0,0.0
4,18,0.0,1.0
5,19,6.0,3.0
6,21,2.0,0.0
7,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,107.0
1,12,Released,0.0
2,15,Released,12.0
3,16,Released,2.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21,22
Status,,,,,,,,
Executing,1.0,1.0,57.0,0.0,1.0,3.0,0.0,0.0
Released,107.0,0.0,12.0,2.0,0.0,6.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21,22
0,Executing,1.0,1.0,57.0,0.0,1.0,3.0,0.0,0.0
1,Released,107.0,0.0,12.0,2.0,0.0,6.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21,22
0,Executing,1.0,1.0,57.0,0.0,1.0,3.0,0.0,0.0
1,Released,107.0,0.0,12.0,2.0,0.0,6.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()